In [10]:
# python version | 3.10.9
# webdriver version | 4.10.0

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random

# 웹 드라이버 초기화
driver = webdriver.Edge()

# 페이지 이동()
driver.get('중고차 사이트 주소 입력')

ind = 1
page = 1  # 진행되고 있는 페이지
max_images = 100  # 크롤링할 이미지 최대 개수
imgs = []

while len(imgs) < max_images:
    try:
        # 페이지 로딩을 대기
        wait = WebDriverWait(driver, 10)
        wait.until(EC.visibility_of_element_located((By.XPATH, f'//*[@id="content"]/div[2]/div/div[2]/div[2]/div[3]/div[4]/div[2]/div[{ind}]/div[1]/a/span/img')))

        # 크롤링할 이미지 클릭하여 상세정보를 받아오고자 함
        driver.find_element(by='xpath', value=f'//*[@id="content"]/div[2]/div/div[2]/div[2]/div[3]/div[4]/div[2]/div[{ind}]/div[1]/a/span/img').click()
        time.sleep(0.3)
        
        # 페이지 전환
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(random.uniform(0.7, 1.5))

        # 첫번째 이미지는 가려진 경우가 대다수여서 다른 이미지 클릭
        driver.find_element(by='xpath', value=f'//*[@id="bx-pager"]/div[1]/a[3]/img').click()
        time.sleep(0.3)

        # 차량 번호와 이미지 주소를 리스트에 입력
        img_alt = driver.find_element(by='xpath', value=f'//*[@id="content"]/div[1]/div[2]/div[2]/div[1]/div[2]/div[1]/strong').text
        img_src = driver.find_element(by='xpath', value=f'//*[@id="btnCarPhotoView"]/li[4]/div[1]/a/img').get_attribute('src')
        imgs.append({
            'alt': img_alt,
            'src': img_src
        })
        driver.close()

        # 페이지 전환(원래 페이지로 돌아옴)
        driver.switch_to.window(driver.window_handles[0])

        # 페이지 정환이 되지 않은경우 (화면에 대상이 보이지 않는경우) 다시한번 페이지 전환
        ## 간혹 전환이 실행되지 않는 경우를 대비함
        if not driver.find_element(by='xpath',
                                   value=f'//*[@id="content"]/div[2]/div/div[2]/div[2]/div[3]/div[4]/div[2]/div[{ind}]/div[1]/a/span/img').is_displayed():
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(3)
            continue

        ind += 1
        print(len(imgs), img_alt, img_src)
        time.sleep(random.uniform(0.3, 0.7))

    except:

        # 다음 페이지로 이동
        driver.find_element(by='xpath',
                            value=f'//*[@id="content"]/div[2]/div/div[2]/div[2]/div[3]/div[5]/a[{page + 1}]').click()
        ind = 1
        page += 1

# 드라이버 종료
driver.quit()


# 크롤링된 이미지 저장

In [ ]:
import urllib.request
import os

# 이미지를 저장할 경로 설정
save_path = f'./car_img_ver.2'

# 경로가 존재않을 경우 경로 생성
if not os.path.exists(save_path):
    os.mkdir(save_path)

for idx, one in enumerate(imgs):
    src = one['src']
    alt = one['alt'].split('\n')[0]
    urllib.request.urlretrieve(src,  f"{save_path}/{idx+1}_{alt}.png")
    print(idx, alt)

print('done')